This notebook was prepared by [Donne Martin](https://github.com/donnemartin). Source and license info is on [GitHub](https://github.com/donnemartin/interactive-coding-challenges).

# Challenge Notebook

## Problem: Given a list of stock prices on each consecutive day, determine the max profits with k transactions.

* [Constraints](#Constraints)
* [Test Cases](#Test-Cases)
* [Algorithm](#Algorithm)
* [Code](#Code)
* [Unit Test](#Unit-Test)
* [Solution Notebook](#Solution-Notebook)

## Constraints

* Is k the number of sell transactions?
    * Yes
* Can we assume the prices input is an array of ints?
    * Yes
* Can we assume the inputs are valid?
    * No
* If the prices are all decreasing and there is no opportunity to make a profit, do we just return 0?
    * Yes
* Should the output be the max profit and days to buy and sell?
    * Yes
* Can we assume this fits memory?
    * Yes

## Test Cases

<pre>
* Prices: None or k: None -> None
* Prices: [] or k == 0 -> []
* Prices: [0, -1, -2, -3, -4, -5]
    * (max profit, list of transactions)
    * (0, [])
* Prices: [2, 5, 7, 1, 4, 3, 1, 3] k: 3
    * (max profit, list of transactions)
    * (10, [Type.SELL day: 7 price: 3, 
            Type.BUY  day: 6 price: 1, 
            Type.SELL day: 4 price: 4, 
            Type.BUY  day: 3 price: 1, 
            Type.SELL day: 2 price: 7, 
            Type.BUY  day: 0 price: 2])
</pre>

## Algorithm

Refer to the [Solution Notebook]().  If you are stuck and need a hint, the solution notebook's algorithm discussion might be a good place to start.

The solution is related to the longest increasing subsequences.
However, instead of selecting for the longest increasing subsequences, we will compute the difference between start and end of these subsequences.
We will then select the top k largest values.
Turn those top k values into Transaction objects

## Code

In [1]:
from enum import Enum  # Python 2 users: Run pip install enum34


class Type(Enum):
    SELL = 0
    BUY = 1


class Transaction(object):

    def __init__(self, type, day, price):
        self.type = type
        self.day = day
        self.price = price

    def __eq__(self, other):
        return self.type == other.type and \
            self.day == other.day and \
            self.price == other.price

    def __repr__(self):
        return str(self.type) + ' day: ' + \
            str(self.day) + ' price: ' + \
            str(self.price)

In [2]:
import numpy as np
class StockTrader(object):

    def find_max_profit(self, prices, k):
        if prices is None or k is None:
            raise TypeError
        if len(prices) == 0 or k == 0:
            return []
        prices = np.asarray(prices)

        prices_shifted = np.insert(prices, 0, 0)
        prices_shifted = prices_shifted[: len(prices_shifted) - 1]

        first_difference = prices - prices_shifted
        first_difference[0] = 0

        positive_movements = first_difference > 0
        if np.count_nonzero(positive_movements) == 0:
            return (0, [])
        
        ix_largest_first_differences = np.flip(np.argsort(first_difference))
        # can take the top k values to get the best single day gains

        ix_neg_movements = np.where(first_difference < 0)[0]
        ix_neg_movements = np.insert(ix_neg_movements, 0, 0)
        ix_neg_movements = np.append(ix_neg_movements, len(prices))

        # split array into: prices[ix_neg_movement[i]:ix_neg_movement[i+1]]
        potential_trades = []
        for i in range(len(ix_neg_movements)):
            end = i+1
            if end >= len(ix_neg_movements):
                end = len(ix_neg_movements) - 1
            if i == end:
                continue
            lb, ub = ix_neg_movements[i], ix_neg_movements[end]
            potential_trades.append((prices[lb: ub], prices[ub - 1] - prices[lb], lb, ub))
        potential_trades = sorted(potential_trades, key=lambda x: x[1], reverse=True)

        # select the top k trades from the set.
        results = []
        profits = 0
        for i in range(k):
            trade = potential_trades[i]
            results.append(Transaction(Type.BUY,  trade[-2], trade[0][0]))
            results.append(Transaction(Type.SELL, trade[-1] - 1, trade[0][-1]))
            profits += trade[1]
        return profits, results

## Unit Test

**The following unit test is expected to fail until you solve the challenge.**

In [3]:
# %load test_max_profit.py
from nose.tools import assert_equal
from nose.tools import assert_raises
from nose.tools import assert_true


class TestMaxProfit(object):

    def test_max_profit(self):
        stock_trader = StockTrader()
        assert_raises(TypeError, stock_trader.find_max_profit, None, None)
        assert_equal(stock_trader.find_max_profit(prices=[], k=0), [])
        prices = [5, 4, 3, 2, 1]
        k = 3
        assert_equal(stock_trader.find_max_profit(prices, k), (0, []))
        prices = [2, 5, 7, 1, 4, 3, 1, 3]
        profit, transactions = stock_trader.find_max_profit(prices, k)
        assert_equal(profit, 10)
        assert_true(Transaction(Type.SELL,
                                day=7,
                                price=3) in transactions)
        assert_true(Transaction(Type.BUY,
                                day=6,
                                price=1) in transactions)
        assert_true(Transaction(Type.SELL,
                                day=4,
                                price=4) in transactions)
        assert_true(Transaction(Type.BUY,
                                day=3,
                                price=1) in transactions)
        assert_true(Transaction(Type.SELL,
                                day=2,
                                price=7) in transactions)
        assert_true(Transaction(Type.BUY,
                                day=0,
                                price=2) in transactions)
        print('Success: test_max_profit')


def main():
    test = TestMaxProfit()
    test.test_max_profit()


if __name__ == '__main__':
    main()

Success: test_max_profit


## Solution Notebook

Review the [Solution Notebook]() for a discussion on algorithms and code solutions.